In [4]:
import numpy as np
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy

# Define 3 traces
trace1 = ["a1", "a2", "a3", "a1", "a3", "a2"]
trace2 = ["a1", "a4", "a3", "a1", "a2", "a1", "a2"]
trace3 = ["a1", "a4", "a2", "a3", "a2","a1", "a2", "a3"]

# construct a adjacency matrix for each trace
# also there is no self-loops in the matrix.  so default the diagnoal to 0.  everything else in matrix should be 1 
def construct_adjacency_matrix(trace):
    # get the unique elements in the trace
    unique_elements = np.unique(trace)
    # create a matrix of zeros
    adj_matrix = np.zeros((len(unique_elements), len(unique_elements)))
    # initialize every element in the matrix to 1 besides the diagonal
    for i in range(len(unique_elements)):
        for j in range(len(unique_elements)):
            if i != j:
                adj_matrix[i][j] = 1
    # # print the matrix with nice formatting to see what it looks like
    # print("Adjacency matrix: \n", adj_matrix)

    # fin the frequency of each pair of elements occuring in the trace. remember order matters we only want to count the pair once and going forward in the trace.
    for i in range(len(trace)-1):
        for j in range(len(unique_elements)):
            for k in range(len(unique_elements)):
                if trace[i] == unique_elements[j] and trace[i+1] == unique_elements[k]:
                    adj_matrix[j][k] += 1
    # # print the matrix with nice formatting to see what it looks like
    # print("Adjacency matrix: \n", adj_matrix)

    # sum the rows after adding the frequency of the pair occuring in the trace to the matrix. use this sum to take row wise probability of the matrix for every row. 
    for i in range(len(unique_elements)):
        adj_matrix[i] = adj_matrix[i]/np.sum(adj_matrix[i])
    return adj_matrix

# construct the adjacency matrix for each trace
adj_matrix_t1 = construct_adjacency_matrix(trace1)
adj_matrix_t2 = construct_adjacency_matrix(trace2)
adj_matrix_t3 = construct_adjacency_matrix(trace3)

# pritn the adjacency matrix for each trace
print("Adjacency matrix for trace1: \n", adj_matrix_t1)
print("Adjacency matrix for trace2: \n", adj_matrix_t2)
print("Adjacency matrix for trace3: \n", adj_matrix_t3)



Adjacency matrix for trace1: 
 [[0.         0.5        0.5       ]
 [0.33333333 0.         0.66666667]
 [0.5        0.5        0.        ]]
Adjacency matrix for trace2: 
 [[0.         0.5        0.16666667 0.33333333]
 [0.5        0.         0.25       0.25      ]
 [0.5        0.25       0.         0.25      ]
 [0.25       0.25       0.5        0.        ]]
Adjacency matrix for trace3: 
 [[0.         0.4        0.2        0.4       ]
 [0.33333333 0.         0.5        0.16666667]
 [0.25       0.5        0.         0.25      ]
 [0.25       0.5        0.25       0.        ]]


In [9]:
# ___ computing kld and jsd from trace matrixes ___

from scipy.special import kl_div
from scipy.sparse import coo_matrix


# create 2 new matrices that will store kld and jsd values for each pair of traces
kld_matrix = np.zeros((3,3))
jsd_matrix = np.zeros((3,3))

# write a function that takes in 2 adjacency matrices and computes the kld and jsd values for each pair of traces
def compute_kld(adj_matrix1, adj_matrix2):
    # Convert them to sparse coordinate format for easier edge extraction
    coo1 = coo_matrix(adj_matrix1)
    coo2 = coo_matrix(adj_matrix2)

    # Create edge-lists
    edges1 = list(zip(coo1.row, coo1.col))
    edges2 = list(zip(coo2.row, coo2.col))

    # Union of nodes in both graphs
    nodes = set(coo1.row).union(set(coo1.col)).union(set(coo2.row)).union(set(coo2.col))

    # Joint set of all possible edges
    joint_edges = [(i, j) for i in nodes for j in nodes]

    # Binary vectors indicating presence/absence of each edge in joint set
    bin_vec1 = np.array([edge in edges1 for edge in joint_edges])
    bin_vec2 = np.array([edge in edges2 for edge in joint_edges])

    # Normalize to form probability distributions
    prob_dist1 = bin_vec1 / np.sum(bin_vec1)
    prob_dist2 = bin_vec2 / np.sum(bin_vec2)

    # Adding small epsilon to avoid division by zero
    epsilon = 1e-10
    prob_dist1 += epsilon
    prob_dist2 += epsilon

    # Ensure the adjusted distributions still sum to 1
    prob_dist1 /= np.sum(prob_dist1)
    prob_dist2 /= np.sum(prob_dist2)

    # Compute KLD
    kld = kl_div(prob_dist1, prob_dist2).sum()

    print('Kullback-Leibler divergence:', kld)
    return kld

# write a function that will run all adjacency matrices through the compute_kld_jsd function
def run_all_traces(adj_matrix_t1, adj_matrix_t2, adj_matrix_t3):
    # run trace1 through the compute_kld function. make sure to store results in the kld_matrix and jsd_matrix. for example kld_matrix[0][1] should be the kld value for trace1 and trace2
    kld_matrix[0][1] = compute_kld(adj_matrix_t1, adj_matrix_t2)
    kld_matrix[0][2] = compute_kld(adj_matrix_t1, adj_matrix_t3)
    kld_matrix[1][2] = compute_kld(adj_matrix_t2, adj_matrix_t3)
    # kld is no symmetric so we need to do this for the other half of the matrix
    kld_matrix[1][0] = compute_kld(adj_matrix_t2, adj_matrix_t1)
    kld_matrix[2][0] = compute_kld(adj_matrix_t3, adj_matrix_t1)
    kld_matrix[2][1] = compute_kld(adj_matrix_t3, adj_matrix_t2)
    # print the kld_matrix
    print("KLD matrix: \n", kld_matrix)

# run all traces through the run_all_traces function
run_all_traces(adj_matrix_t1, adj_matrix_t2, adj_matrix_t3)

Kullback-Leibler divergence: 0.6931471669422313
Kullback-Leibler divergence: 0.6931471669422313
Kullback-Leibler divergence: 0.0
Kullback-Leibler divergence: 9.923898546726695
Kullback-Leibler divergence: 9.923898546726695
Kullback-Leibler divergence: 0.0
KLD matrix: 
 [[0.         0.69314717 0.69314717]
 [9.92389855 0.         0.        ]
 [9.92389855 0.         0.        ]]


In [10]:
# ---- test adjacency matrix is being constructed correctly ----
import numpy as np
from scipy.spatial.distance import jensenshannon
from scipy.stats import entropy

# Define 3 traces
trace1 = ["a1", "a2", "a3", "a1", "a3", "a2"]

# construct a adjacency matrix for each trace
# also there is no self-loops in the matrix.  so default the diagnoal to 0.  everything else in matrix should be 1 
def construct_adjacency_matrix(trace):
    # get the unique elements in the trace
    unique_elements = np.unique(trace)
    # create a matrix of zeros
    adj_matrix = np.zeros((len(unique_elements), len(unique_elements)))
    # initialize every element in the matrix to 1 besides the diagonal
    for i in range(len(unique_elements)):
        for j in range(len(unique_elements)):
            if i != j:
                adj_matrix[i][j] = 1
    # print the matrix with nice formatting to see what it looks like
    print("Adjacency matrix: \n", adj_matrix)

    # fin the frequency of each pair of elements occuring in the trace. remember order matters we only want to count the pair once and going forward in the trace.
    for i in range(len(trace)-1):
        for j in range(len(unique_elements)):
            for k in range(len(unique_elements)):
                if trace[i] == unique_elements[j] and trace[i+1] == unique_elements[k]:
                    adj_matrix[j][k] += 1
    # print the matrix with nice formatting to see what it looks like
    print("Adjacency matrix: \n", adj_matrix)

    # sum the rows after adding the frequency of the pair occuring in the trace to the matrix. use this sum to take row wise probability of the matrix for every row. 
    for i in range(len(unique_elements)):
        adj_matrix[i] = adj_matrix[i]/np.sum(adj_matrix[i])
    return adj_matrix

# pritn the adjacency matrix for each trace
print("Adjacency matrix for trace1: \n", construct_adjacency_matrix(trace1))

Adjacency matrix: 
 [[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
Adjacency matrix: 
 [[0. 2. 2.]
 [1. 0. 2.]
 [2. 2. 0.]]
Adjacency matrix for trace1: 
 [[0.         0.5        0.5       ]
 [0.33333333 0.         0.66666667]
 [0.5        0.5        0.        ]]


In [ ]:
# also there is no self-loops in the matrix.  so default the diagnoal to 0.  everything else in matrix should be 1 
# add the frequcny of the pair occuring in the trace to the matrix
# sum the rows after adding the frequency of the pair occuring in the trace to the matrix
# use this sum to take row wise probability of the matrix for every row. 
# make a matrix like this for every trace
# then between the matrix compute KLD and JSD

# ___ computing kld and jsd from trace matrixes ___
# need to take union of all the pairs from the traces we are comparing
# create 2 new matrix that is comprise of the similarity between the traces using KLD and JSD

# ___ MST ___
# run MST on the matrix